# VacationPy
----

1. Create a heat map that displays the humidity for every city from WeatherPy dataframe.
2. Add Nearest Hotel to Lat/Lon searched after adding conditions.


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from config import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load: CSV
# Loading CSV to not re-run data everytime. 
# Used as a checkpoint
cities_csv_df = pd.read_csv('../WeatherPy/output/random_cities.csv', encoding='utf-8')

df_main = cities_csv_df.copy()
df_main=df_main.drop(['Unnamed: 0'], axis=1)
df_main.head()

,City_Name,Lat,Lon,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,mataura,-46.19,168.86,58.95,74.0,83.0,2.93,NZ,1.588478e+09
1,butaritari,3.07,172.79,83.35,81.0,32.0,14.79,KI,1.588477e+09
2,san quintin,30.48,-115.95,64.45,77.0,0.0,14.63,MX,1.588477e+09
3,georgetown,5.41,100.34,89.60,70.0,20.0,5.82,MY,1.588477e+09
4,homer,59.64,-151.55,51.80,53.0,40.0,8.05,US,1.588477e+09


In [3]:
#df_narrowed = df_main[['Lat','Lon','Max_Temp','Humidity', 'Cloudiness', 'Wind_Speed']]

In [4]:
# Drop rows with Nan values
df_main.dropna(subset = (["Lat", 'Max_Temp', 'Wind_Speed', 'Cloudiness']), inplace=True)
df_main.shape

(576, 9)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Single out Lat,Lon values to combine into list
lat_cords = [i for i in df_main['Lat']]
lon_cords = [i for i in df_main['Lon']]
# Bring out humit values. Used for weights in gmaps
humid_weights = df_main['Humidity'].tolist()

In [6]:
# Go through two lists to create [(lat,lng)] for coordinate values. 
# Used in gmaps
coordinates = []
for i,j in zip(lat_cords, lon_cords):
    coords = (i,j)
    coordinates.append(coords)

In [7]:
# Configure GMAPS
# Customize the size of the figure
# Use the Lat and Lng as locations and Humidity as the weight.
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(coordinates, weights=humid_weights, 
                                 dissipating=False, max_intensity=70,
                                 point_radius=3)

#markers = gmaps.marker_layer(coordinates)
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

#### Importing screenshot for GitHub reference. Not showing original gmaps plot online.
#### Humidity Heatmap
![gmap](screens/gmaps2.jpg)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.

    * A max temperature lower than 80 degrees but higher than 70.

    * Wind speed less than 10 mph.

    * Zero cloudiness.

* Drop any rows will null values. (dropped when creating df_main)

In [8]:
# Make a copy of cleaned df_main
df_ideal_weather = df_main.copy()

In [9]:
# Narrow down cities to fit weather conditions
df_ideal_weather = df_ideal_weather.query('70 < Max_Temp < 80 & Wind_Speed < 10 & Cloudiness == 0')
df_ideal_weather

,City_Name,Lat,Lon,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
60,geraldton,-28.77,114.60,78.80,24.0,0.0,9.17,AU,1.588477e+09
68,carnarvon,-24.87,113.63,78.80,65.0,0.0,9.17,AU,1.588478e+09
113,lazaro cardenas,17.96,-102.20,75.49,77.0,0.0,1.95,MX,1.588478e+09
135,la orilla,17.98,-102.23,74.88,77.0,0.0,1.63,MX,1.588478e+09
204,vila velha,-20.33,-40.29,73.00,94.0,0.0,2.24,BR,1.588477e+09
226,coahuayana,18.73,-103.68,72.93,64.0,0.0,1.52,MX,1.588478e+09
239,linxia,35.60,103.21,75.78,10.0,0.0,3.53,CN,1.588478e+09
245,praya,14.92,-23.51,73.40,53.0,0.0,8.05,CV,1.588478e+09
381,tessalit,20.20,1.01,76.12,16.0,0.0,5.59,ML,1.588478e+09
388,saint-pierre,-21.34,55.48,73.40,60.0,0.0,6.93,RE,1.588478e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Copy ideal weather df and name it hotel_df
hotel_df = df_ideal_weather.copy()

In [11]:
# Add 'Hotel Name' columns to hotel_df
hotel_df['Hotel Name'] = ''

In [12]:
# Display new empty 'Hotel Name' column
hotel_df

,City_Name,Lat,Lon,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name
60,geraldton,-28.77,114.60,78.80,24.0,0.0,9.17,AU,1.588477e+09,
68,carnarvon,-24.87,113.63,78.80,65.0,0.0,9.17,AU,1.588478e+09,
113,lazaro cardenas,17.96,-102.20,75.49,77.0,0.0,1.95,MX,1.588478e+09,
135,la orilla,17.98,-102.23,74.88,77.0,0.0,1.63,MX,1.588478e+09,
204,vila velha,-20.33,-40.29,73.00,94.0,0.0,2.24,BR,1.588477e+09,
226,coahuayana,18.73,-103.68,72.93,64.0,0.0,1.52,MX,1.588478e+09,
239,linxia,35.60,103.21,75.78,10.0,0.0,3.53,CN,1.588478e+09,
245,praya,14.92,-23.51,73.40,53.0,0.0,8.05,CV,1.588478e+09,
381,tessalit,20.20,1.01,76.12,16.0,0.0,5.59,ML,1.588478e+09,
388,saint-pierre,-21.34,55.48,73.40,60.0,0.0,6.93,RE,1.588478e+09,


In [13]:
# Loop through hotel_df -- Goal: Add hotel name to 'Hotel Name' column in hotel_df. 
# Loop using lat/lon columns in hotel_df
# set up a parameters dictionary
nearby_hotel = {
    "keyword": "Hotel",
    "radius": 5000,
    "key": g_key
}

# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    # Retreive values needed to display in DF.
    get_lat = row['Lat']
    get_lon = row['Lon']
    get_city = row['City_Name']
    get_country = row['Country']
    hotel_coords = (get_lat, get_lon)
    
    # add keyword to params dict
    nearby_hotel['location'] = (get_lat,get_lon)

    # Assemble url and make API request
    print(f"Retrieving Results for Index {index}: {hotel_coords}.")
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}".format(get_lat,get_lon)
    response = requests.get(url, params=nearby_hotel).json()
    
    try:
        # extract results
        hotel_name = response['results'][0]['name']
        print(f"Found closest Hotel to Index {index}.  Hotel Name: {hotel_name}.")
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
        
    except (IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "None Found"
        print(f"No Hotels found within {nearby_hotel['radius']}m radius. Added 'None Found' value. Moving on.")
        
    print("----- Completed -------")

Retrieving Results for Index 60: (-28.77, 114.6).
Found closest Hotel to Index 60.  Hotel Name: Ocean Centre Hotel.
----- Completed -------
Retrieving Results for Index 68: (-24.87, 113.63).
Found closest Hotel to Index 68.  Hotel Name: Hospitality Carnarvon.
----- Completed -------
Retrieving Results for Index 113: (17.96, -102.2).
Found closest Hotel to Index 113.  Hotel Name: Hotel de Casablanca.
----- Completed -------
Retrieving Results for Index 135: (17.98, -102.23).
Found closest Hotel to Index 135.  Hotel Name: Hotel de Casablanca.
----- Completed -------
Retrieving Results for Index 204: (-20.33, -40.29).
Found closest Hotel to Index 204.  Hotel Name: Quality Suites Vila Velha.
----- Completed -------
Retrieving Results for Index 226: (18.73, -103.68).
Found closest Hotel to Index 226.  Hotel Name: Hotel Los Arcos.
----- Completed -------
Retrieving Results for Index 239: (35.6, 103.21).
Found closest Hotel to Index 239.  Hotel Name: Linxia Hotel.
----- Completed -------
Retr

In [14]:
# hotel_df after API loop adding hotel name added to 'Hotel Name' in hotel_df
hotel_df

,City_Name,Lat,Lon,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name
60,geraldton,-28.77,114.60,78.80,24.0,0.0,9.17,AU,1.588477e+09,Ocean Centre Hotel
68,carnarvon,-24.87,113.63,78.80,65.0,0.0,9.17,AU,1.588478e+09,Hospitality Carnarvon
113,lazaro cardenas,17.96,-102.20,75.49,77.0,0.0,1.95,MX,1.588478e+09,Hotel de Casablanca
135,la orilla,17.98,-102.23,74.88,77.0,0.0,1.63,MX,1.588478e+09,Hotel de Casablanca
204,vila velha,-20.33,-40.29,73.00,94.0,0.0,2.24,BR,1.588477e+09,Quality Suites Vila Velha
226,coahuayana,18.73,-103.68,72.93,64.0,0.0,1.52,MX,1.588478e+09,Hotel Los Arcos
239,linxia,35.60,103.21,75.78,10.0,0.0,3.53,CN,1.588478e+09,Linxia Hotel
245,praya,14.92,-23.51,73.40,53.0,0.0,8.05,CV,1.588478e+09,Hotel VIP Praia
381,tessalit,20.20,1.01,76.12,16.0,0.0,5.59,ML,1.588478e+09,None Found
388,saint-pierre,-21.34,55.48,73.40,60.0,0.0,6.93,RE,1.588478e+09,LODGE PALMAE


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City_Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [16]:
# Add marker layer ontop of heat map
# Display figure
# Add Market layer, displaying Hotel information, to map.
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

#### Importing screenshot for GitHub reference. Not showing original gmaps plot online.
![gmaps](screens/gmaps1.jpg)